In [1]:
'''
Step:1 

n=number of min hashes
b=number of bands
r=number of min hash functions for each band
1. Create a hashmap(dict) of (26+10)*(26+10) possible values(00...zz) and broadcast.
2. Generate n=b*r hash functions and broadcast. To make these functions. create b*r arrays of 1296 elements(-1 and 1)
2.1 For each document, create shingles(bi grams).
2.2 Perform following step n times for each of the hash functions
    Apply hash fuction to each ngram(or a subset of shingles-random selection for a large doc) and 
    store the minhash for that iteration
1.3 Now you have a list of minhashes for a document. Split it into b bands of r minhashes each
    and apply a hash function/concat to generate a min hash for the array of r minhashes
    Now emit(hash(list(minhash)),band id),doc id
1.4 group all doc ids by hash(list(minhash)
1.5 these are the matching documents.
1.6 Thrshold t~ (1/b) pow (1/r). So tune accordingly

'''

##Read the file into a RDD and then create ngrams/shingles hash each string to integer 
##using CRC32
##Do not need random selection of shingles since the word list is not quite large
##In the mapper, apply hashCRC to each shingle and then  apply n hash functions and create a list of minhashes.
##Now in the same  mapper, Prepare n/x =1000/50 =20 bands and emit (band id,hash of sub-list of min hashes),DOC ID
## In combiner group all doc ids by hash codes and band id



'\nStep:1 \n\nn=number of min hashes\nb=number of bands\nr=number of min hash functions for each band\n1. Create a hashmap(dict) of (26+10)*(26+10) possible values(00...zz) and broadcast.\n2. Generate n=b*r hash functions and broadcast. To make these functions. create b*r arrays of 1296 elements(-1 and 1)\n2.1 For each document, create shingles(bi grams).\n2.2 Perform following step n times for each of the hash functions\n    Apply hash fuction to each ngram(or a subset of shingles-random selection for a large doc) and \n    store the minhash for that iteration\n1.3 Now you have a list of minhashes for a document. Split it into b bands of r minhashes each\n    and apply a hash function/concat to generate a min hash for the array of r minhashes\n    Now emit(hash(list(minhash)),band id),doc id\n1.4 group all doc ids by hash(list(minhash)\n1.5 these are the matching documents.\n1.6 Thrshold t~ (1/b) pow (1/r). So tune accordingly\n\n'

In [2]:
'''def hashCRC(ngram):
    import binascii 
    crc = binascii.crc32(ngram) & 0xffffffff
    return crc

def ngramsMinHash(line,tuple_size,num_buckets,num_bands,h1,max_prime,coeff):
  
  words=line.split(",")
  arr=[]
 
  import re
  name=re.sub(r'\s+', '', words[1]+words[2]).lower()
  
  for i in xrange((len(name)-tuple_size+1)):
    #print (name[i]+name[i+1],words[0])
    j=1
    tempGram=""
    while(j<tuple_size):
        tempGram=tempGram+name[i+j]
        j+=1
        
        
    arr.append(hashCRC(name[i]+tempGram))
  
  num_hashes=num_buckets/num_bands
  minHash=[[] for i in xrange(num_bands)]
 
  for i in xrange(num_buckets):
        hash_each_iter=[]
       
        for val in arr:
            hash_each_iter.append(h1(val,max_prime,coeff[0][i],coeff[1][i]))
        
        #print min(hash_each_iter)
        minHash[int(i/num_hashes)].append(str(min(hash_each_iter)))
       
    
  ##Now apply another hash function to each stripe of each band and emit  and hash (value, bandid), doc id  
  ## This will be reduced and similar pairs emitted
 
  print name,words[0]
  for i in xrange(len(minHash)):
     #yield (i,hash(tuple(minHash[i]))),words[0]
     yield (i,''.join(minHash[i])),words[0]   
'''
  

'def hashCRC(ngram):\n    import binascii \n    crc = binascii.crc32(ngram) & 0xffffffff\n    return crc\n\ndef ngramsMinHash(line,tuple_size,num_buckets,num_bands,h1,max_prime,coeff):\n  \n  words=line.split(",")\n  arr=[]\n \n  import re\n  name=re.sub(r\'\\s+\', \'\', words[1]+words[2]).lower()\n  \n  for i in xrange((len(name)-tuple_size+1)):\n    #print (name[i]+name[i+1],words[0])\n    j=1\n    tempGram=""\n    while(j<tuple_size):\n        tempGram=tempGram+name[i+j]\n        j+=1\n        \n        \n    arr.append(hashCRC(name[i]+tempGram))\n  \n  num_hashes=num_buckets/num_bands\n  minHash=[[] for i in xrange(num_bands)]\n \n  for i in xrange(num_buckets):\n        hash_each_iter=[]\n       \n        for val in arr:\n            hash_each_iter.append(h1(val,max_prime,coeff[0][i],coeff[1][i]))\n        \n        #print min(hash_each_iter)\n        minHash[int(i/num_hashes)].append(str(min(hash_each_iter)))\n       \n    \n  ##Now apply another hash function to each stripe of e

In [16]:
import binascii 
import re
from collections import Counter
from pyspark.mllib.linalg import SparseVector

def bigram_map():
    
    a=['0','1','2','3','4','5','6','7','8','9','a','b','c','d','e','f','g','h','i','j','k','l','m','n','o','p','q','r','s','t','u','v','w','x','y','z']
    c=0
    d={}
    for i in a:
        for j in a:
           d[i+j]=c
           c+=1
    return d

def ngrams_min_hash(line,tuple_size,coeffs,num_bands):
  
  words=line.split(",",1)
  arr=[]
  dict_bigram=bigram_map()
  
  name_temp=re.sub(r'\s+', '', words[1])
  name=re.sub("[^\w]",'',name_temp).lower()
 
  for i in xrange((len(name)-tuple_size+1)):
    j=1
    tempGram=""
    while(j<tuple_size):
        tempGram=tempGram+name[i+j]
        j+=1
        
        
    arr.append(dict_bigram[name[i]+tempGram])
  
  sv=SparseVector(1296,dict(Counter(arr)))
  arr=[]
  for i in xrange(num_bands):
        yield words[0],i,sv  
            
           

 

In [17]:
'''
import numpy as np
def gen_hash_coeffs(max_prime):
    
    a1=np.random.randint(0, max_prime, size=1)[0]
    b1=np.random.randint(0, max_prime, size=1)[0]
    a2=np.random.randint(0, max_prime, size=1)[0]
    b2=np.random.randint(0, max_prime, size=1)[0]
    #return[a1,b1,a2,b2]
    

'''

'\nimport numpy as np\ndef gen_hash_coeffs(max_prime):\n    \n    a1=np.random.randint(0, max_prime, size=1)[0]\n    b1=np.random.randint(0, max_prime, size=1)[0]\n    a2=np.random.randint(0, max_prime, size=1)[0]\n    b2=np.random.randint(0, max_prime, size=1)[0]\n    #return[a1,b1,a2,b2]\n    \n\n'

In [18]:
from  pyspark.mllib.random import RandomRDDs

def gen_hash_coeffs_1(num_buckets,num_bigrams,num_bands,seed):
    #coeff=[[] for j in xrange(num_bands)]
    coeff=[None]*num_bands
    r=num_buckets/num_bands
    #arr=[[] for j in xrange(r)]
    arr=[None]*r
    for i in xrange(num_buckets):
        nRDD= RandomRDDs.normalRDD(sc, num_bigrams, seed=seed+i).map(lambda val: str(-1) if val < 0 else str(1)).collect()
        j=nRDD
        idx= i%(r)
     
        arr[idx]=(idx+1,j)
        if i%r ==r-1:
            coeff[i/r]=arr
            arr=[None]*r
    return coeff


In [19]:
len(coeff)

NameError: name 'coeff' is not defined

In [20]:
def LSH(line,coeff):
    r=(coeff.value)[line[1]]
    hash_str=''
    for i in xrange(len(r)):
        hash_str=hash_str+str(1 if line[2].dot(r[i][1]).sum() > 0 else 0)
    return hash_str,line[0]

        
        
        
    
    

In [21]:
import time
#time.ctime() # 'Mon Oct 18 13:35:29 2010'
time.strftime('%l:%M%p %S seconds %Z on %b %d, %Y')

' 9:58AM 52 seconds CDT on Apr 10, 2017'

In [98]:

#worked for small list NamesLSH
#num_buckets=1000
#num_bands=50
##worked better for NamesLSH
#num_bands=15 
#num_buckets=210
num_bands=20 
num_buckets=400
coeffs=sc.broadcast(gen_hash_coeffs_1(num_buckets,1296,num_bands,100))
#coeffs
#coeffs
#print coeffs.collect()
#print coeffs
##Create a dictionary of bigrams:
#print coeffs.map(lambda sv:).collect()

#rdd_name=sc.textFile("/Users/anant/Documents/Projects/SingleView/us-names-1.txt")
rdd_name=sc.textFile("/Users/anant/Documents/Projects/SingleView/100.txt")
#rdd_name=sc.textFile("/Users/anant/Documents/Projects/SingleView/NamesLSH.txt")
#sc.broadCast(coeffs)
#match_buckets=rdd_name.flatMap(lambda line: ngrams_min_hash(line,2,num_buckets,num_bands,coeffs)).collect()
#match_buckets=rdd_name.flatMap(lambda line: ngrams_min_hash(line,2,num_buckets,num_bands,coeffs)).combineByKey(lambda value:[value],lambda x,value:x+[value],lambda x,value:x+value).filter(lambda (x,y): len(y)>= 2).collect()
doc_vector=rdd_name.flatMap(lambda line: ngrams_min_hash(line,2,coeffs,num_bands)).map(lambda line:LSH(line,coeffs)).combineByKey(lambda value:[value],lambda x,value:x+[value],lambda x,value:x+value).filter(lambda (x,y): len(y)>= 2).map(lambda val: (','.join(val[1]),val[0])).reduceByKey(lambda x,y: x)
#matches=doc_vector.map(lambda line:coeffs.map(lambda coeff: ((line[0],coeff[0]/num_bands),(coeff[0]%num_bands,(1 if line[1].dot(coeff[1]).sum() > 0 else 0))))).collect()
#biGrams=rdd_name.flatMap(lambda line: ngramsMinHash(line,2,50,25,h1,h2,max_prime)).collect()

In [99]:
x=sqlContext.createDataFrame(doc_vector, ["matches", "mKey"])
x.write.format("com.mongodb.spark.sql.DefaultSource").option("uri","mongodb://localhost:27017/").option("database","spark").option("collection","test").mode("overwrite").save()
matches= doc_vector.collect()
print matches

[(u'16,17', '10110000010101110010'), (u'1,2', '00101010101001110011'), (u'18,19', '10100001010101110101')]


In [81]:
import time
#time.ctime() # 'Mon Oct 18 13:35:29 2010'
time.strftime('%l:%M%p %S seconds %Z on %b %d, %Y')

' 1:28PM 19 seconds CDT on Apr 10, 2017'

In [82]:
d={}
my_set=set()
#with open('/Users/anant/Documents/Projects/SingleView/us-names-1.txt') as f:
with open('/Users/anant/Documents/Projects/SingleView/100.txt') as f:
    for line in f:
        #print line
        vals=line.split(",",1)
        #print vals[1].strip()
        d[vals[0].strip()]=vals[1].strip()

for i in xrange(len(matches)):
    
    print "Match-"+str(i+1)
    names=matches[i][0].split(",")
    
    for j in xrange(len(names)):
        my_set.add(d[str(names[j])])
        print d[names[j]]
    print "\n\n"
    
print len(my_set)

Match-1
aaron, bill r
aaron, billy



Match-2
abbott, erik t
abbott, eric



Match-3
abbott, erik t
abbott, j



Match-4
aaron, billy
aaron, b



Match-5
abbott, jerry l
abbott, jerry w



Match-6
aaronson, cary
abbate, gary w



Match-7
aaron, bill r
aaron, b



Match-8
abbott, eric
abbott, jerry w



10


In [84]:
matches.saveToMongoDB('mongodb://localhost:27017/spark.test.output')

In [ ]:
spark.